# Import Dependancies

In [88]:
import tensorflow as tf
import pandas as pd
import keras

In [89]:
keras.__version__

'2.11.0'

In [90]:
tf.__version__

'2.11.0'

In [91]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# Read in the Data 

In [92]:
rawData = pd.read_excel("Full-Data-Set-UnderOver-2020-21.xlsx")
rawData = rawData.sample(frac=1)

In [93]:
rawData.describe()

,Unnamed: 0,GP,W,L,W_PCT,MIN,FGM,FGA,FG_PCT,FG3M,...,BLK_RANK.1,BLKA_RANK.1,PF_RANK.1,PFD_RANK.1,PTS_RANK.1,PLUS_MINUS_RANK.1,Score,Home-Team-Win,OU,OU-Cover
count,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,...,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000,16358.000000
mean,8178.500000,42.738538,22.225883,20.512654,0.513536,48.353754,37.952763,83.493654,0.454603,8.192689,...,15.307984,15.168480,15.353344,15.477809,15.275095,15.036007,204.782186,0.591148,204.598392,0.515344
std,4722.292187,24.007672,15.430625,13.403097,0.183969,0.325391,2.318313,3.831431,0.019487,2.528331,...,8.621444,8.633495,8.640666,8.612438,8.633568,8.664541,22.006642,0.491637,21.729718,0.526352
min,0.000000,1.000000,0.000000,0.000000,0.000000,48.000000,28.000000,67.300000,0.339000,0.700000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,22.000000,0.000000,1.000000,0.000000
25%,4089.250000,22.000000,10.000000,9.000000,0.382000,48.100000,36.400000,80.900000,0.442000,6.300000,...,8.000000,8.000000,8.000000,8.000000,8.000000,7.000000,190.000000,0.000000,194.500000,0.000000
50%,8178.500000,43.000000,19.000000,19.000000,0.520000,48.300000,37.800000,83.400000,0.454000,7.900000,...,15.000000,15.000000,15.000000,16.000000,15.000000,15.000000,204.000000,1.000000,203.500000,1.000000
75%,12267.750000,64.000000,32.000000,29.000000,0.645000,48.500000,39.300000,86.100000,0.467000,9.900000,...,23.000000,23.000000,23.000000,23.000000,23.000000,22.000000,219.000000,1.000000,214.000000,1.000000
max,16357.000000,82.000000,72.000000,69.000000,1.000000,55.500000,50.000000,100.000000,0.571000,20.000000,...,30.000000,30.000000,30.000000,30.000000,30.000000,30.000000,329.000000,1.000000,1955.500000,2.000000


## Clean the Data

In [94]:
filteredData = rawData.drop(["Unnamed: 0"], axis = 1)
filteredData = filteredData.drop(["Score"], axis = 1)
filteredData = filteredData.drop(["GP"], axis = 1)
filteredData = filteredData.drop(["GP.1"], axis = 1)
filteredData = filteredData.drop(["Home-Team-Win"], axis = 1)
filteredData = filteredData.drop(["TEAM_NAME"], axis = 1)
filteredData = filteredData.drop(["Date"], axis = 1)
filteredData = filteredData.drop(["MIN", "W", "W.1", "L", "L.1", "PLUS_MINUS", "PLUS_MINUS.1", "PLUS_MINUS_RANK", "PLUS_MINUS_RANK.1", "W_RANK", "W_RANK.1", "L_RANK", "L_RANK.1"], axis = 1)
filteredData = filteredData.drop(["MIN.1"], axis = 1)
filteredData = filteredData.drop(["MIN_RANK.1"], axis = 1)
filteredData = filteredData.drop(["MIN_RANK"], axis = 1)
filteredData = filteredData.drop(["TEAM_NAME.1"], axis = 1)
filteredData = filteredData.drop(["Date.1"], axis = 1)
filteredData = filteredData.drop(["OU"], axis = 1)
filteredData = filteredData.drop(["OU-Cover"], axis = 1)
filteredData = filteredData.drop(["GP_RANK"], axis = 1)
filteredData = filteredData.drop(["GP_RANK.1"], axis = 1)

# Train the Model

#### Get the outputs

In [95]:
output = rawData['Home-Team-Win']

In [96]:
#get 80% for training 20% for testing
print(len(filteredData))
test_len = int(len(filteredData) * 0.8)
x_train = filteredData.take(range(test_len))
x_test = filteredData.take(range(test_len, len(filteredData)))
y_train = output.take(range(test_len))
y_test = output.take(range(test_len, len(filteredData)))

16358


In [97]:
test_len = int(len(filteredData) * 0.9)
x_train = filteredData.take(range(test_len))
x_test = filteredData.take(range(test_len, len(filteredData)))
y_train = output.take(range(test_len))
y_test = output.take(range(test_len, len(filteredData)))

In [98]:
import datetime
%load_ext tensorboard
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [99]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(170, activation=tf.nn.relu6))
model.add(tf.keras.layers.Dropout(rate=.25))
model.add(tf.keras.layers.Dense(150, activation=tf.nn.relu6))
model.add(tf.keras.layers.Dense(75, activation=tf.nn.relu6))
model.add(tf.keras.layers.Dropout(rate=.15))
model.add(tf.keras.layers.Dense(30, activation=tf.nn.relu6))
model.add(tf.keras.layers.Dense(15, activation=tf.nn.relu6))
model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax, name="output_layer"))


In [100]:
model.build( x_train.shape )
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_4 (Flatten)         (14722, 84)               0         
                                                                 
 dense_20 (Dense)            (14722, 170)              14450     
                                                                 
 dropout_8 (Dropout)         (14722, 170)              0         
                                                                 
 dense_21 (Dense)            (14722, 150)              25650     
                                                                 
 dense_22 (Dense)            (14722, 75)               11325     
                                                                 
 dropout_9 (Dropout)         (14722, 75)               0         
                                                                 
 dense_23 (Dense)            (14722, 30)              

In [101]:
from keras.optimizers import Adam
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
model.compile(Adam(learning_rate=0.0005), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [106]:
model.fit(x_train, y_train, epochs=60, validation_split=0.30, batch_size=3_000, verbose=2, shuffle=True, workers=15, use_multiprocessing=True, callbacks=[tensorboard_callback])

Epoch 1/60
4/4 - 0s - loss: 0.5840 - accuracy: 0.6941 - val_loss: 0.5733 - val_accuracy: 0.7046 - 138ms/epoch - 35ms/step
Epoch 2/60


/home/damon/Desktop/Model/lib/python3.10/site-packages/keras/engine/data_adapter.py:1696: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


4/4 - 0s - loss: 0.5822 - accuracy: 0.6978 - val_loss: 0.5748 - val_accuracy: 0.7021 - 93ms/epoch - 23ms/step
Epoch 3/60
4/4 - 0s - loss: 0.5838 - accuracy: 0.6954 - val_loss: 0.5742 - val_accuracy: 0.7021 - 91ms/epoch - 23ms/step
Epoch 4/60
4/4 - 0s - loss: 0.5836 - accuracy: 0.6969 - val_loss: 0.5703 - val_accuracy: 0.7048 - 86ms/epoch - 22ms/step
Epoch 5/60
4/4 - 0s - loss: 0.5798 - accuracy: 0.7016 - val_loss: 0.5775 - val_accuracy: 0.6984 - 96ms/epoch - 24ms/step
Epoch 6/60
4/4 - 0s - loss: 0.5808 - accuracy: 0.7003 - val_loss: 0.5712 - val_accuracy: 0.7055 - 97ms/epoch - 24ms/step
Epoch 7/60
4/4 - 0s - loss: 0.5814 - accuracy: 0.6973 - val_loss: 0.5705 - val_accuracy: 0.7068 - 93ms/epoch - 23ms/step
Epoch 8/60
4/4 - 0s - loss: 0.5767 - accuracy: 0.7009 - val_loss: 0.5735 - val_accuracy: 0.7043 - 83ms/epoch - 21ms/step
Epoch 9/60
4/4 - 0s - loss: 0.5841 - accuracy: 0.6899 - val_loss: 0.5705 - val_accuracy: 0.7073 - 83ms/epoch - 21ms/step
Epoch 10/60
4/4 - 0s - loss: 0.5801 - accur

In [103]:
todayPredictions = model.predict([x_test])

52/52 [==============================] - 0s 715us/step


In [104]:
import numpy as np
print(len(y_test))
print(len(x_test))
win = 0
for i in range (0,len(todayPredictions)):
    try:
        if np.argmax(todayPredictions[i]) == y_test[i]:
            win += 1
    except Exception as e:
        exit(1)

print(win / len(todayPredictions))

1636
1636
0.0550122249388753


# Save the Model

In [105]:
import datetime
model.save(f"model_{datetime.datetime.utcnow()}")

INFO:tensorflow:Assets written to: model_2022-12-14 01:16:26.534008/assets
